In [1]:
!pip install hyperopt

In [13]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  
  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_date', (X_train, y_train)),
      callbacks=[tensorboard_callback]
  )
  
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(num_classes, activation='softmax'),                   
  ])


In [10]:
model = get_cnn_v3(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 6s 23ms/step - loss: 2.2776 - accuracy: 0.3800 - val_loss: 0.6659 - val_accuracy: 0.8358
Epoch 2/5
272/272 [==============================] - 6s 21ms/step - loss: 0.6523 - accuracy: 0.8027 - val_loss: 0.1577 - val_accuracy: 0.9583
Epoch 3/5
272/272 [==============================] - 6s 22ms/step - loss: 0.3218 - accuracy: 0.9020 - val_loss: 0.0678 - val_accuracy: 0.9844
Epoch 4/5
272/272 [==============================] - 6s 21ms/step - loss: 0.2154 - accuracy: 0.9349 - val_loss: 0.0471 - val_accuracy: 0.9874
Epoch 5/5
272/272 [==============================] - 6s 21ms/step - loss: 0.1661 - accuracy: 0.9512 - val_loss: 0.0305 - val_accuracy: 0.9911


0.9625850340136054

In [0]:
def func_obj(params):
  
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0,
  )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [25]:
space = {
    'batch size': hp.quniform('batch size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch size': 150.0, 'dropout_cnn_block_one': 0.3078289627467506, 'dropout_cnn_block_three': 0.407271487614553, 'dropout_cnn_block_two': 0.30964527260601604, 'dropout_dense_block_one': 0.5176586074019216}
accuracy=0.9700680375099182
{'batch size': 170.0, 'dropout_cnn_block_one': 0.34458657624958905, 'dropout_cnn_block_three': 0.3036196061446653, 'dropout_cnn_block_two': 0.4506584602067814, 'dropout_dense_block_one': 0.683417392634184}
accuracy=0.9553288221359253
{'batch size': 130.0, 'dropout_cnn_block_one': 0.4468806785806696, 'dropout_cnn_block_three': 0.42671505858633696, 'dropout_cnn_block_two': 0.3935330205252401, 'dropout_dense_block_one': 0.38920640958685304}
accuracy=0.9424036145210266
{'batch size': 170.0, 'dropout_cnn_block_one': 0.3763908581558456, 'dropout_cnn_block_three': 0.35101980577833897, 'dropout_cnn_block_two': 0.36150380695392814, 'dropout_dense_block_one': 0.44044719991913367}
accuracy=0.9614512324333191
{'batch size': 140.0, 'dropout_cnn_block_one': 0.4006292821